In [22]:
%load_ext tensorboard


import numpy as np
import pandas as pd
import pandas_profiling
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
import os, datetime, glob
from shutil import copyfile

reSize=(224,224)
batchSize=32
h5FileName='ResNet50-V2-simpons.h5'




The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
train=keras.preprocessing.image.ImageDataGenerator(rescale=1./255,zoom_range=0.1,width_shift_range=0.05,height_shift_range=0.05,horizontal_flip=True)
valid=keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

trainGenerator=train.flow_from_directory('train',target_size=reSize,batch_size=batchSize,class_mode='categorical', color_mode='rgb')
validGenerator=valid.flow_from_directory('valid',target_size=reSize,batch_size=batchSize,class_mode='categorical', color_mode='rgb')
# for cls, idx in trainGenerator.class_indices.items():
#     print('Class {} = {}'.format(idx, cls))


Found 13605 images belonging to 20 classes.
Found 5795 images belonging to 20 classes.


In [23]:
def create_model():
    ResNet=keras.applications.ResNet50V2(include_top=False, weights='imagenet',input_shape=(reSize[0],reSize[1],3))
    return models.Sequential(
        [    
            ResNet,
            layers.GlobalAveragePooling2D(),
            layers.Dense(512,activation="relu"),
            layers.Dense(20, activation="softmax"),

        ]
    )




In [ ]:
model = create_model()
model.compile(optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True), loss="categorical_crossentropy", metrics=["accuracy"])
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.summary()
model.fit_generator(trainGenerator,steps_per_epoch=trainGenerator.samples//batchSize,epochs=30,validation_data=validGenerator,validation_steps=validGenerator.samples//batchSize,callbacks=[tensorboard_callback],)


Epoch 1/5
425/425 [==============================] - 391s 920ms/step - loss: 0.0040 - accuracy: 0.9988 - val_loss: 0.0730 - val_accuracy: 0.9786
Epoch 2/5
425/425 [==============================] - 390s 919ms/step - loss: 0.0022 - accuracy: 0.9993 - val_loss: 0.0738 - val_accuracy: 0.9795
Epoch 3/5
425/425 [==============================] - 389s 916ms/step - loss: 0.0040 - accuracy: 0.9986 - val_loss: 0.1619 - val_accuracy: 0.9617
Epoch 4/5
425/425 [==============================] - 391s 919ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.1873 - val_accuracy: 0.9522
Epoch 5/5
387/425 [==========================>...] - ETA: 30s - loss: 0.0072 - accuracy: 0.9978

In [33]:
model.save(h5FileName)


In [30]:
test=keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
testGenerator=test.flow_from_directory('test',target_size=reSize,batch_size=1,class_mode='categorical',shuffle=False, color_mode='rgb')


Found 990 images belonging to 1 classes.


In [66]:
predict=model.predict_generator(testGenerator)

In [5]:
model.evaluate_generator(validGenerator)

[0.09882673180374575, 0.9758412]

In [31]:
def submussion():
    predict=model.predict_generator(testGenerator)
    predict= np.argmax(predict, axis=1)
    labels = (trainGenerator.class_indices)
    label = dict((v,k) for k,v in labels.items())
    predictions = [label[k] for k in predict]
    idList=[]
    for filename in testGenerator.filenames:
        idList.append(int(filename[2:-4]))
    sub=pd.DataFrame(idList)
    sub.columns = ['id']
    sub['character']=predictions
    sub.sort_values(by=['id']).to_csv("Submussion(9).csv", index=False)


In [32]:
submussion()

In [13]:
look=np.argmax(k, axis=1)

In [44]:
predict= np.argmax(predict, axis=1)

idList=[]
for filename in testGenerator.filenames:
    idList.append(filename[2:])

In [45]:
fileList=[]
for index,val in enumerate(predict):
    if val != 2:
        fileList.append(idList[index])

In [76]:
predict2= np.argmax(predict, axis=1)


In [ ]:
predict2=[]
for i in predict:
    predict2.append(np.argmax(i, axis=1))

In [ ]:
testGenerator.filepaths[i].split('/')

In [78]:
len(predict2)

19548

In [90]:
trainGenerator.class_indices['abraham_grampa_simpson']

0

In [ ]:
from shutil import copyfile
sorce = os.path.join("RAW_DATA", dirname, file)
destination = os.path.join("noise", dirname, file)

In [99]:
predict=model.predict_generator(testGenerator)
predict2= np.argmax(predict, axis=1)
for index,val in enumerate(predict2):
     if val != trainGenerator.class_indices[testGenerator.filepaths[index].split('/')[-2]]:
        sorce = os.path.join("RAW_DATA",testGenerator.filenames[index])
        destination = os.path.join("noise", testGenerator.filenames[index])
        copyfile(sorce, destination)
        os.remove(sorce)

In [26]:
model.layers[0].summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________